In [ ]:
from re import X
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
mnist=fetch_openml('mnist_784',version=1)
m=len(mnist["data"])
X = mnist["data"] / 255.0   # 정규화
Y = mnist["target"].astype(np.int32)
X=X.dropna()
Y=Y.dropna()
X=(np.array(X)).reshape(m,784)
Y=(np.array(Y)).reshape(m,1)
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=1004)
class LogisticRegression:
    def __init__(self,lr=0.5,epochs=100):
        self.lr=lr
        self.epochs=epochs
        self.theta=None
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def compute_loss(self,Y,Y_hat):
        m=len(Y)
        return -1/m*np.sum(Y*np.log(Y_hat+1e-8)+(1-Y)*np.log(1-Y_hat+1e-8))
    def fit(self,X,Y):
        m,n=X.shape
        X=np.hstack((np.ones((m, 1)), X))
        self.theta=np.zeros(n+1).reshape(n+1,1)
        losses=[]
        for i in range(self.epochs):
            z=np.dot(X,self.theta)
            Y_hat=self.sigmoid(z)
            gradient=(1/m)*X.T.dot(Y_hat-Y) #gradient의 크기를 지정을 안해줌 세타는 해줬는데 회귀에서도
            self.theta=self.theta-self.lr*gradient
            loss=self.compute_loss(Y,Y_hat)
            losses.append(loss)
        return losses
    def predict_probability(self,X): #Y_hat 출력
        m=X.shape[0]
        X=np.hstack((np.ones((m, 1)), X))
        return self.sigmoid(np.dot(X,self.theta))

    def predict(self,X,threshold=0.5):
        probability=self.predict_probability(X)
        return (probability>=threshold).astype(int)
models=[]
for digit in range(10):
    Y_train_binary=(Y_train==(digit)).astype(int)
    model=LogisticRegression(lr=0.5,epochs=100)
    model.fit(X_train,Y_train_binary)
    models.append(model)
train_scores = np.zeros((X_train.shape[0], 10))
for digit, model in enumerate(models):
    prob = model.predict_probability(X_train)
    train_scores[:, digit] = prob.ravel()
Y_train_pred = np.argmax(train_scores, axis=1)  # shape: (n_samples,)
Y_train_true = Y_train.ravel()
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

cm = confusion_matrix(Y_train_true, Y_train_pred)
print("Confusion Matrix:\n", cm)

precision = precision_score(Y_train_true, Y_train_pred, average='macro')
recall = recall_score(Y_train_true, Y_train_pred, average='macro')
f1 = f1_score(Y_train_true, Y_train_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
 [[4585    0   16   11   10   13   44    6   67    2]
 [   0 5355   24   17    1   37    8    9   81    8]
 [  42   80 4111   87  107    7  118   94  176   42]
 [  37   38  138 4343    7  160   51   65  127   68]
 [  11   30   35    8 4304    7   54   11   70  256]
 [ 107   58   35  278   97 3427  106   36  239  109]
 [  48   34   42    5   30   74 4500    3   49    2]
 [  36   84   85   21   75    8    7 4508   20  166]
 [  34  151   68  161   35  140   40   26 4082   88]
 [  42   38   40   96  217   43    5  185   64 4178]]
Precision: 0.8849772078801609
Recall: 0.8838301445676005
F1 Score: 0.8837815276233947
